In [2]:
from cafu.metadata import path
import pandas as pd

In [3]:
path_camp = path('datalake')+'/jogos_ids/espanha/2012-2013.csv'
df = pd.read_csv(path_camp)

In [9]:
list(df['time_casa'].drop_duplicates().sort_values())==list(df['time_visitante'].drop_duplicates().sort_values())

True

In [54]:
times = list(df['time_casa'].append(df['time_visitante']).drop_duplicates().sort_values())

/tmp/ipykernel_6437/3303466267.py:1: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  times = list(df['time_casa'].append(df['time_visitante']).drop_duplicates().sort_values())


In [47]:
df['time_casa']

0           Rayo Vallecano
1            Real Zaragoza
2            Celta de Vigo
3                  Granada
4                Barcelona
              ...         
335             Sevilla FC
336         Rayo Vallecano
337          Celta de Vigo
338    Deportivo La Coruña
339          Athletic Club
Name: time_casa, Length: 340, dtype: object

In [63]:
def _teste_partidas_campeonato(df, qt_jogos_rodada, qt_partidas_campeonato, campeonato, temporada, runtime_str):
    """
    Método interno da biblioteca cafu.
    Testa as condições de sucesso da função partidas_campeonato
    """
    
    times = list(df['time_casa'].append(df['time_visitante']).drop_duplicates().sort_values())
    jogos_faltantes = {}
    for i, t in enumerate(times):
        index_filter = df['time_casa']==t
        times_enfrentados = list(df[index_filter]['time_visitante'].drop_duplicates().sort_values())
        times_copy = times[:i]+times[(i+1):]
        f = list(set(times_copy).difference(times_enfrentados))
        if len(f) > 0:
            jogos_faltantes[t] = f
    
    success = True
    if df.shape[0] != qt_partidas_campeonato:
        success = False
        logging.error(f"ERROR etl.data_lake.partidas_campeonato.partidas_campeonato: "
                      f"Unexpected number of lines in returned dataframe (expected amount of league matches): "
                      f"<returned>={df.shape[0]}, <expected>={qt_partidas_campeonato}. "
                      f"Return <min_id_campeonato>={min(ids)}, <max_id_campeonato>={max(ids)}, <excedentes>={excedentes}. "
                      f"<campeonato>={campeonato}, <temporada>={temporada}, <qt_jogos_rodada>={qt_jogos_rodada}. "
                      f"runtime = {runtime_str}")
    if len(times)!=2*qt_jogos_rodada:
        success = False
        logging.error(f"ERROR etl.data_lake.partidas_campeonato.partidas_campeonato: "
                      f"Unexpected number of teams: "
                      f"<returned>={len(times)} <expected>={2*qt_jogos_rodada}. "
                      f"Return <min_id_campeonato>={min(ids)}, <max_id_campeonato>={max(ids)}, <excedentes>={excedentes}. "
                      f"<campeonato>={campeonato}, <temporada>={temporada}, <qt_jogos_rodada>={qt_jogos_rodada}. "
                      f"runtime = {runtime_str}")
    if len(jogos_faltantes)>0:
        success = False
        logging.error(f"ERROR etl.data_lake.partidas_campeonato.partidas_campeonato: "
                      f"Not all returned teams faced each other: "
                      f"<jogos_faltantes>={jogos_faltantes}. "
                      f"Return <min_id_campeonato>={min(ids)}, <max_id_campeonato>={max(ids)}, <excedentes>={excedentes}. "
                      f"<campeonato>={campeonato}, <temporada>={temporada}, <qt_jogos_rodada>={qt_jogos_rodada}. "
                      f"runtime = {runtime_str}")
    if success:
        logging.info(f"SUCCESS etl.data_lake.partidas_campeonato.partidas_campeonato: Function executed successfully. "
                 f"Return <min_id_campeonato>={min(ids)}, <max_id_campeonato>={max(ids)}. "
                 f"<campeonato>={campeonato}, <temporada>={temporada}, <qt_jogos_rodada>={qt_jogos_rodada}. "
                 f"runtime = {runtime_str}")

In [ ]:
if df.shape[0] == qt_partidas_campeonato:
    logging.info(f"SUCCESS etl.data_lake.partidas_campeonato.partidas_campeonato: Function executed successfully. "
                 f"Return <min_id_campeonato>={min(ids)}, <max_id_campeonato>={max(ids)}. "
                 f"<campeonato>={campeonato}, <temporada>={temporada}, <qt_jogos_rodada>={qt_jogos_rodada}. "
                 f"runtime = {runtime_str}")
else:
    logging.error(f"ERROR etl.data_lake.partidas_campeonato.partidas_campeonato: "
                  f"Unexpected number of lines in returned dataframe (expected amount of league matches): "
                  f"<returned>={df.shape[0]}, <expected>={qt_partidas_campeonato}. "
                  f"Return <min_id_campeonato>={min(ids)}, <max_id_campeonato>={max(ids)}, <excedentes>={excedentes}. "
                  f"<campeonato>={campeonato}, <temporada>={temporada}, <qt_jogos_rodada>={qt_jogos_rodada}. "
                  f"runtime = {runtime_str}")

In [66]:
len(_teste_partidas_campeonato(df, 10))

/tmp/ipykernel_6437/436250022.py:3: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  times = list(df['time_casa'].append(df['time_visitante']).drop_duplicates().sort_values())


2

In [28]:
times = list(df['time_casa'].drop_duplicates().sort_values())
times

['Athletic Club',
 'Atlético de Madrid',
 'Barcelona',
 'Celta de Vigo',
 'Deportivo La Coruña',
 'Espanyol',
 'Getafe',
 'Granada',
 'Levante',
 'Mallorca',
 'Málaga',
 'Osasuna',
 'Rayo Vallecano',
 'Real Betis',
 'Real Madrid',
 'Real Sociedad',
 'Real Zaragoza',
 'Sevilla FC',
 'Valencia']

In [62]:
#for t in times:
t = 'Athletic Club'
i = 0

index_filter = df['time_casa']==t
times_enfrentados = list(df[index_filter]['time_visitante'].drop_duplicates().sort_values())
times_copy = times[:i]+times[(i+1):]
list(set(times_copy).difference(times_enfrentados))

[]

In [57]:
for i, t in enumerate(times):
    print(i,t)

0 Athletic Club
1 Atlético de Madrid
2 Barcelona
3 Celta de Vigo
4 Deportivo La Coruña
5 Espanyol
6 Getafe
7 Granada
8 Levante
9 Mallorca
10 Málaga
11 Osasuna
12 Rayo Vallecano
13 Real Betis
14 Real Madrid
15 Real Sociedad
16 Real Zaragoza
17 Sevilla FC
18 Valencia


In [39]:
times_copy

['Athletic Club',
 'Atlético de Madrid',
 'Celta de Vigo',
 'Deportivo La Coruña',
 'Espanyol',
 'Getafe',
 'Granada',
 'Levante',
 'Mallorca',
 'Málaga',
 'Osasuna',
 'Rayo Vallecano',
 'Real Betis',
 'Real Madrid',
 'Real Sociedad',
 'Real Zaragoza',
 'Sevilla FC',
 'Valencia']

In [40]:
times_enfrentados

['Athletic Club',
 'Atlético de Madrid',
 'Celta de Vigo',
 'Deportivo La Coruña',
 'Espanyol',
 'Getafe',
 'Granada',
 'Levante',
 'Mallorca',
 'Málaga',
 'Osasuna',
 'Rayo Vallecano',
 'Real Betis',
 'Real Madrid',
 'Real Zaragoza',
 'Sevilla FC',
 'Valencia']